Before we begin, let's execute the below cell to display information about the CUDA driver and GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above. If all goes well, you should see some output returned below the grey cell.

In [ ]:
!nvidia-smi

## Exercise 5

###  Learning objectives

The **goal** of this lab is to:

- Learn how to inspect the application's kernels with Nsight Compute
- Learn how to execute rules inside the Nsight Computer profiler and find bottlenecks
- Learn how to add baselines and compare results/reports

We do not intend to cover:

- OpenACC Programming models
- Advanced Optimization techniques in detail

As mentioned earlier on, Nsight Compute and Nsight Systems each serve a different purpose in profiling and their functionalities are different. In previous exercises, we inspected the timelines, measured activity durations, and tracked CPU events via the Nsight Systems profiler. The purpose of this exercise is to get familiar with Nsight Compute tool. This tool provides access to kernel-level analysis using GPU performance metrics.

We first profile the GPU application and identify certain areas in the code, where they don't behave as expected. Then we isolate those kernels and profile them via Nsight Compute. 

**Understand and analyze** the code present at:

[OpenACC Code](../source_code/lab5/miniWeather_openacc.cpp) 

Open the downloaded file for inspection. Once done, **Compile** the code with `make` and **Profile** it with `nsys`.

In [ ]:
!cd ../source_code/lab5 && make clean && make

Now, **Profile** the code with Nsight System CLI:

In [ ]:
!cd ../source_code/lab5 && nsys profile -t nvtx,openacc --stats=true --force-overwrite true -o miniWeather_5 ./miniWeather

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab5/miniWeather_5.nsys-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight System UI locally. As you can see in the example output, the initialization looks very expensive and kernels are very small that the GPU compute part of the problem is very small. Check how much time (what percentage) is spent in each relative to the time it takes to run the code. 

<img src="images/5_init.png" width="80%">

From the timeline view, inspect the less efficient kernel. Let's take the most expensive kernel and take a closer look and see what the Nsight Compute recommends us to do.

<img src="images/cexer5.png" width="80%">

Now, **Profile** the application via Nsight Compute CLI (`ncu`): 

In [ ]:
!cd ../source_code/lab5 && ncu --set full -k regex:compute_tendencies_x --launch-skip 10 --launch-count 1 -f -o miniWeather1 ./miniWeather

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab5/miniWeather1.ncu-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight Compute UI. This tool has a lot of sections that focus on different areas of the GPU and presents them all on one page. 

**Note:** If you don't specify a specific kernel name when profiling, all kernels will be profiled and this will slow down the profiling time.

<img src="images/ccompute.png">

The GPU Speed Of Light section shows less than 1% Compute (SM) Throughput. As you can see from the example output below, Nsight Compute profiler suggests looking at the "Launch Statistics" section because the kernel grid is too small to fill the available resources on the GPU. 

We previously discussed Amdahl's law in the first exercise. It is very important to understand the relation between the problem size and computational performance as this can determine the amount of speedup and benefit you would get by parallelizing on GPU. Due to the small problem size (`nx_glob`, `nz_glob` , and `sim_time` in this example), most of the computation is dominated by the initialization and there is not enough work/computation to make it suitable for GPU. Let's run the application with different values `nx_glob`, `nz_glob` , and `sim_time` and profile the same kernel (`nx_glob` = 400 , `nz_glob`= 200 , and `sim_time`= 100). 

In [ ]:
!cd ../source_code/lab5 && make clean && make

In [ ]:
!cd ../source_code/lab5 && ncu --set full -k regex:compute_tendencies_x --launch-skip 100 --launch-count 1 -f -o miniWeather2 ./miniWeather 400 200 100

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab5/miniWeather2.ncu-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight Compute UI. 

**Diff the reports**

Open both reports via the Nsight Compute UI. From the top of the first report, click on the *Add Baseline*, then do the same for the second tab which is the second report. Have a look at the expected output:

<img src="images/c2compute.png">

You can now compare the two reports and see how changes you made to the cell size, affected specific metrics by looking at each section. By increasing the cell size, we increased the Compute and Memory Throughput by 35% and 26% respectively. 


Now, let's look at the roofline chart. High-level overview of the utilization for compute and memory resources of the GPU is presented as a roofline chart. We can see that increasing cells size resulted in the performance to get closer to the roof lines. 

<img src="images/roofline.png">

However, the kernel is still small to utilized the GPU and Compute and Memory are still less than 50% utilized. The Speed of Light Throughput section, gives a high-level overview of the throughput for compute and memory resources of the GPU For each unit. Based on this information we can find the performance limiters and catagorize them into four possible combinations:

- Compute Bound: SM>50% & Mem<50%
- Bandwidth Bound: SM<50% & Mem>50%
- Latency Bound: SM<50% & Mem<50%
- Compute and Bandwidth Bound : SM>50% & Mem>50%


According to the roofline, this kernel is fp64 bound, and we should consider using 32-bit precision floating point operations to improve its performance. 

The detailed memory workload analysis section shows all the data traffic between various stages of the GPU and what your kernel is actually transferring. This section suggests that we need to look at the memory access pattern in the code as the load/store pattern is not optimal. The solution is to try and minimize how many cache lines need to be accessed per memory  request. 


This is out of scope of this tutorial but  you can have a look the algorithm and see if you can change anything to do more work per memory access.

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommended you go to your browser's File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well. You can also execute the following cell block to create a zip file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
cd ..
rm -f _profiler_files.zip
zip -r _profiler_files.zip *

**After** executing the above zip command, you should be able to download and save the zip file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../_profiler_files.zip) and choosing <mark>Save Link As</mark>. 

-----

# <div style="text-align: center ;border:3px; border-style:solid; border-color:#FF0000  ; padding: 1em">[HOME](../_start_profiling.ipynb)</div>

-----

# Links and Resources

[OpenACC API Guide](https://www.openacc.org/sites/default/files/inline-files/OpenACC%20API%202.6%20Reference%20Guide.pdf)

[NVIDIA Nsight Compute](https://docs.nvidia.com/nsight-compute/index.html)

[CUDA Toolkit Download](https://developer.nvidia.com/cuda-downloads)

**NOTE**: To be able to see the Nsight Compute profiler output, please download Nsight Compute latest version from [here](https://developer.nvidia.com/nsight-compute).

Don't forget to check out additional [OpenACC Resources](https://www.openacc.org/resources) and join our [OpenACC Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

--- 

## Licensing 

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).